In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
requests.packages.urllib3.disable_warnings()
import urllib.request
import ssl
import re
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')
import calendar
months = {'Января':'01', 'Февраля':'02','Марта':'03','Апреля':'04','Мая':'05','Июня':'06','Июля':'07',
          'Августа':'08','Сентября':'09','Октября':'10','Ноября':'11','Декабря':'11'}


In [7]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

baseurl = 'https://reestr.minsvyaz.ru'

In [8]:
#поиск количества страниц c отображением по 100
def find_n():
    req = requests.get('https://reestr.minsvyaz.ru/reestr/?show_count=100', verify=False).text
    res = BeautifulSoup(req, 'lxml').find('div', class_='page_nav_area').find_all('a', class_='nav_item')
    return int(res[3].text)+1

In [21]:
for i in range(1, find_n()):
    # Создание ссылок для всех страниц поиска на сайте реестра (с отображением по 100)
    url = baseurl + '/reestr/?PAGEN_1=' + str(i) + '&show_count=100'
    response = urllib.request.urlopen(url)
    print(url)
    # поиск ссылок на страницы с юридической информации о ПО
    links = re.findall('<a href=\"(/reestr/\d*/)\"', str(response.read()))
    with open('links.txt', mode='a', encoding='utf-8') as myfile:
        for line in links:
            print(baseurl + line, file=myfile)
    myfile.close

https://reestr.minsvyaz.ru/reestr/?PAGEN_1=1&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=2&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=3&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=4&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=5&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=6&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=7&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=8&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=9&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=10&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=11&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=12&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=13&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=14&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=15&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=16&show_count=100
https://reestr.minsvyaz.ru/reestr

In [22]:
# константные значения
links = "links.txt"
# количество уникальных названий ПО
with open(links) as f:
    num = sum(1 for _ in f)

# создание словаря для передачи его в последствии датафрейму в пандас
values = {'Название по': [], 'Вид организации':[],'Название организации': [], 'Все продукты организации':[], 'ИНН': [], 'Сайт': [], 'Альтернатив': [],  'Класс по': [], 'Дата регистрации':[], 'Рег.номер ПО':[],
         'Дата решения':[],'Решение':[],'Ссылка на приказ':[]}

values['Название по'] = ['' for element in range(num)]
values['Название организации'] = ['' for element in range(num)]
values['ИНН'] = ['' for element in range(num)]
values['Сайт'] = ['' for element in range(num)]
values['Альтернатив'] = ['' for element in range(num)]
values['Класс по'] = ['' for element in range(num)]
values['Вид организации'] = ['' for element in range(num)]
values['Все продукты организации'] = ['' for element in range(num)]
values['Дата регистрации'] = ['' for element in range(num)]
values['Рег.номер ПО'] = ['' for element in range(num)]
values['Дата решения'] = ['' for element in range(num)]
values['Решение'] = ['' for element in range(num)]
values['Ссылка на приказ'] = ['' for element in range(num)]


In [24]:
f = open(links)
i = 0
for url in f:
    print(url)
    print(str(i) + ' of ' + str(num))
    req = requests.get(url, verify=False).text
    soup = BeautifulSoup(req, 'lxml')
    # 1 название ПО
    soft = soup.find('h1', id='pagetitle').text
    values['Название по'][i] = soft
    # 2 название организации
    try:
        orgname = soup.find_all('a', title=re.compile("Все продукты"))
    except AttributeError:
        orgname = ""
        pass
    values['Название организации'][i] = '::'.join([i.text.strip() for i in orgname])
    # 3 ИНН ^(\d{10}|\d{12})$
    try:
        inn = soup.find_all('div', text=re.compile("(\d{10}\s|\d{12}\s)"))
    except AttributeError:
        inn = ''
        pass
    #инн может быть несколько, например в случае 4 правообладателей-людей, поэтому инн в списке
    values['ИНН'][i] = '::'.join([i.text.strip() for i in inn])
    # 4 альтернативное название организации
    try:
        altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div').text
        altname = re.sub('Альтернативные наименования:', '', altname)
    except AttributeError:
        altname = ""
        pass
    values['Альтернатив'][i] = altname.strip()
    # 5 Класс по
    try:
        softclass = soup.find('span', text=re.compile("Класс ПО:")).find_parent('div').find('font').text
        softclass = re.sub('Класс ПО:', '', softclass)
        softclass = softclass.strip()
    except AttributeError:
        softclass = ""
        pass
    values['Класс по'][i] = softclass
    # 6 сайт производителя
    try:
        site = soup.find('span', text=re.compile("Сайт производителя:")).find_parent('div').find('a').get('href')
    except AttributeError:
        site = ""
        pass
    values['Сайт'][i] = site
    # 7 вид организации
    try:
        type_of_organization = soup.find("h5", attrs={ "style" : "font-size: 20px; margin-bottom: 12px;"}).text
    except AttributeError:
        type_of_organization = ""
        pass
    values['Вид организации'][i] = type_of_organization
    # 8 все продукты организации
    try:
        all_products = soup.find('a', title='Все продукты организации').get('href')
    except AttributeError:
        all_products = ""
        pass
    values['Все продукты организации'][i] = 'https://reestr.digital.gov.ru' + all_products
    # 9 дата регистрации
    try:
        date_of_registration = soup.find('span', text=re.compile("Дата регистрации:")).find_parent('div').text
        temp = re.sub('Дата регистрации:', '', date_of_registration).strip()
        temp2 = ' '.join([x for x in temp.split(' ') if x!='']).split(' ')
        temp3 = '.'.join([temp2[0],months.get(temp2[1]),temp2[2]])
    except AttributeError:
        temp3 = ""
        pass
    values['Дата регистрации'][i] = temp3
    # 10 регистрационный номер ПО
    try:
        reg_number = soup.find('span', text=re.compile("Рег. номер ПО:")).find_parent('div').text
        r = re.sub('Рег. номер ПО:', '', reg_number).strip()
    except AttributeError:
        r = ""
        pass
    values['Рег.номер ПО'][i] = r
    # 11 дата решения
    try:
        date_of_decision = soup.find('span', text=re.compile("Дата решения")).find_parent('div').text
        tempp = re.sub('Дата решения уполномоченного органа:', '', date_of_decision).strip()
        tempp2 = ' '.join([x for x in tempp.split(' ') if x!='']).split(' ')
        tempp3 = '.'.join([tempp2[0],months.get(tempp2[1]),tempp2[2]])
    except AttributeError:
        tempp3 = ""
        pass
    values['Дата решения'][i] = tempp3
    # 12 решение
    try:
        decision = soup.find('span', text=re.compile("Решение уполномоченного")).find_parent('div').text
    except AttributeError:
        decision = ""
        pass
    values['Решение'][i] = re.sub('Решение уполномоченного органа:', '', decision).strip()
    # 13 ссылка на приказ минкомсвязи
    try:
        link = soup.find('span', text=re.compile("Ссылка на приказ")).find_parent('div').find('a').get('href')
    except AttributeError:
        link = ""
        pass
    values['Ссылка на приказ'][i] = link
    i = i + 1
    if i>130:
        break


https://reestr.minsvyaz.ru/reestr/58191/

0 of 8716
https://reestr.minsvyaz.ru/reestr/58193/

1 of 8716
https://reestr.minsvyaz.ru/reestr/58192/

2 of 8716
https://reestr.minsvyaz.ru/reestr/61242/

3 of 8716
https://reestr.minsvyaz.ru/reestr/61245/

4 of 8716
https://reestr.minsvyaz.ru/reestr/61250/

5 of 8716
https://reestr.minsvyaz.ru/reestr/61248/

6 of 8716
https://reestr.minsvyaz.ru/reestr/61251/

7 of 8716
https://reestr.minsvyaz.ru/reestr/61252/

8 of 8716
https://reestr.minsvyaz.ru/reestr/61253/

9 of 8716
https://reestr.minsvyaz.ru/reestr/61255/

10 of 8716
https://reestr.minsvyaz.ru/reestr/61256/

11 of 8716
https://reestr.minsvyaz.ru/reestr/61257/

12 of 8716
https://reestr.minsvyaz.ru/reestr/61258/

13 of 8716
https://reestr.minsvyaz.ru/reestr/61259/

14 of 8716
https://reestr.minsvyaz.ru/reestr/61260/

15 of 8716
https://reestr.minsvyaz.ru/reestr/61261/

16 of 8716
https://reestr.minsvyaz.ru/reestr/61262/

17 of 8716
https://reestr.minsvyaz.ru/reestr/61263/

18 of 8716
htt

In [11]:
#преобразование словаря в датафрейм
df = pd.DataFrame(values)
#запись датафрейма в csv-файл с кодировкой utf-8
df.to_csv('result.csv', encoding = 'utf-8', index = False)

In [38]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61265/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div').find('br')
altname.text

''

In [20]:
with open(links) as f:
    numm = len(f.readlines())
numm

69728

In [23]:
num

8716